In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()

from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('1.02.+Multiple+linear+regression.csv')
data.head()

,SAT,"Rand 1,2,3",GPA
0,1714,1,2.40
1,1664,3,2.52
2,1760,3,2.54
3,1685,3,2.74
4,1693,2,2.83


In [3]:
data.describe()

,SAT,"Rand 1,2,3",GPA
count,84.000000,84.000000,84.000000
mean,1845.273810,2.059524,3.330238
std,104.530661,0.855192,0.271617
min,1634.000000,1.000000,2.400000
25%,1772.000000,1.000000,3.190000
50%,1846.000000,2.000000,3.380000
75%,1934.000000,3.000000,3.502500
max,2050.000000,3.000000,3.810000


In [4]:
x = data[['SAT', 'Rand 1,2,3']]
y = data['GPA']
reg = LinearRegression()
reg.fit(x,y)

LinearRegression()

In [5]:
reg.coef_

array([ 0.00165354, -0.00826982])

In [6]:
reg.intercept_

0.29603261264909486

In [7]:
reg.score(x,y)

0.4066811952814282

$R^2_{adj.} = 1 - (1-R^2)*\frac{n-1}{n-p-1}$

In [9]:
x.shape

(84, 2)

In [11]:
r2 = reg.score(x,y)
n = x.shape[0]     # 84
p = x.shape[1]     # 2
adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
adjusted_r2

0.39203134825134

FEATURE SELECTION

In [12]:
from sklearn.feature_selection import f_regression
f_regression(x,y)

(array([56.04804786,  0.17558437]), array([7.19951844e-11, 6.76291372e-01]))

In [13]:
f_statistics = f_regression(x,y)[0]
p_values = f_regression(x,y)[1]    #Only p values we need
p_values

array([7.19951844e-11, 6.76291372e-01])

In [14]:
p_values.round(3)

array([0.   , 0.676])

CREATING A SUMMARY TABLE

In [18]:
#reg_summary = pd.DataFrame(data=['SAT','Rand 1,2,3'], columns=['Features'])
reg_summary = pd.DataFrame(data=x.columns.values, columns=['Features'])
reg_summary

,Features
0,SAT
1,"Rand 1,2,3"


In [22]:
reg_summary['Coefficients'] = reg.coef_
reg_summary['p_values'] = p_values.round(3)
reg_summary

,Features,Coefficients,7.19951843656382e-11,0.6762913721375519,p_values
0,SAT,0.001654,0.0,0.676,0.000
1,"Rand 1,2,3",-0.008270,0.0,0.676,0.676


If the p value is more than 0.5 then that does not contribute ti the model and must removed. i.e Rand 1,2,3 in this case

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#This object will be used to scale the data. (Formula is written in the book)
scaler.fit(x)

StandardScaler()

In [27]:
x_scaled = scaler.transform(x)
x_scaled

array([[-1.26338288, -1.24637147],
       [-1.74458431,  1.10632974],
       [-0.82067757,  1.10632974],
       [-1.54247971,  1.10632974],
       [-1.46548748, -0.07002087],
       [-1.68684014, -1.24637147],
       [-0.78218146, -0.07002087],
       [-0.78218146, -1.24637147],
       [-0.51270866, -0.07002087],
       [ 0.04548499,  1.10632974],
       [-1.06127829,  1.10632974],
       [-0.67631715, -0.07002087],
       [-1.06127829, -1.24637147],
       [-1.28263094,  1.10632974],
       [-0.6955652 , -0.07002087],
       [ 0.25721362, -0.07002087],
       [-0.86879772,  1.10632974],
       [-1.64834403, -0.07002087],
       [-0.03150724,  1.10632974],
       [-0.57045283,  1.10632974],
       [-0.81105355,  1.10632974],
       [-1.18639066,  1.10632974],
       [-1.75420834,  1.10632974],
       [-1.52323165, -1.24637147],
       [ 1.23886453, -1.24637147],
       [-0.18549169, -1.24637147],
       [-0.5608288 , -1.24637147],
       [-0.23361183,  1.10632974],
       [ 1.68156984,

REGRESSION WITH SCALED FEATURES

In [28]:
reg = LinearRegression()
reg.fit(x_scaled,y)

LinearRegression()

In [30]:
reg.coef_

array([ 0.17181389, -0.00703007])

In [31]:
reg.intercept_

3.330238095238095

CREATING A SUMMARY TABLE

In [34]:
reg_summary = pd.DataFrame([['Bias'],['SAT'],['Rand 1,2,3']], columns=['Features'])
reg_summary['Weights'] = reg.intercept_, reg.coef_[0], reg.coef_[1]

In [35]:
reg_summary

,Features,Weights
0,Bias,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


MAKING PREDICTIONS WITH STANDARDIZED COEFFICIENTS (WEIGHTS)

In [36]:
new_data = pd.DataFrame(data=[[1700,2],[1800,1]], columns=['SAT', 'Rand 1,2,3'])
new_data

,SAT,"Rand 1,2,3"
0,1700,2
1,1800,1


In [37]:
reg.predict(new_data)

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


array([295.39979563, 312.58821497])

In [38]:
new_data_scaled = scaler.transform(new_data)
new_data_scaled

array([[-1.39811928, -0.07002087],
       [-0.43571643, -1.24637147]])

In [40]:
reg.predict(new_data_scaled)

array([3.09051403, 3.26413803])

WHAT IF WE REMOVED THE Rand 1,2,3 variable. THEORATICALLY NOTHING SHOULD CHANGE AS THAT WAS INSIGNIFICANT AND DID NOT ADD ANYTHING TO THE MODEL.

In [41]:
reg_simple = LinearRegression()
x_simple_matrix = x_scaled[:,0].reshape(-1,1)
reg_simple.fit(x_simple_matrix,y)

LinearRegression()

In [43]:
reg_simple.predict(new_data_scaled[:,0].reshape(-1,1))

array([3.08970998, 3.25527879])